In [1]:
from zad11 import connect, load_table, process

In [2]:
spark = connect()

Connecting to Spark...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 17:29:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/10 17:29:57 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
load_table(spark, "/charts_genre_popularity")

Loading '/charts_genre_popularity' into table 'charts_genre_popularity'...


DataFrame[region: string, date: string, pop: string, rap: string, rock: string, edm: string, hip_hop: string, trap_latino: string, reggaeton: string, electropop: string, dance_pop: string, pop_rap: string, musica_mexicana: string, trap: string, modern_rock: string, classic_rock: string, uk_pop: string, k_pop: string, tropical_house: string, melodic_rap: string, canadian_pop: string, modern_bollywood: string]

In [4]:
load_table(spark, "/input/charts_daily_popularity.csv")

Loading '/input/charts_daily_popularity.csv' into table 'charts_daily_popularity'...


DataFrame[region: string, date: string, popularity: string]

In [5]:
load_table(spark, "/input/charts_daily_sum.csv", header=["region", "date", "streams"])

Loading '/input/charts_daily_sum.csv' into table 'charts_daily_sum'...


DataFrame[region: string, date: string, streams: string]

In [6]:
load_table(spark, "/input/daily_country_weather.csv")

Loading '/input/daily_country_weather.csv' into table 'daily_country_weather'...


DataFrame[country: string, date: string, temperature_c: string, precipitation_mm: string]

In [7]:
load_table(spark, "/input/wdi_interpolated.csv")

Loading '/input/wdi_interpolated.csv' into table 'wdi_interpolated'...


DataFrame[country: string, date: string, rural_population_percent: string, fertility_rate: string, gdp_per_capita_usd: string, mobile_subscriptions_per_100: string, refugee_population_promille: string]

In [8]:
process(spark, "output", """
SELECT
    cgp.region AS country,
    cgp.date,
    cds.streams AS daily_streams,
    cdp.popularity AS daily_popularity,
    dcw.temperature_c AS temperature_c,
    dcw.precipitation_mm AS precipitation_mm,
    wdi.rural_population_percent AS `wdi:rural_population_percent`,
    wdi.fertility_rate AS `wdi:fertility_rate`,
    wdi.gdp_per_capita_usd AS `wdi:gdp_per_capita_usd`,
    wdi.mobile_subscriptions_per_100 AS `wdi:mobile_subscriptions_per_100`,
    wdi.refugee_population_promille AS `wdi:refugee_population_promille`,
    cgp.pop AS `genre:pop`,
    cgp.rap AS `genre:rap`,
    cgp.rock AS `genre:rock`,
    cgp.edm AS `genre:edm`,
    cgp.hip_hop AS `genre:hip_hop`,
    cgp.trap_latino AS `genre:trap_latino`,
    cgp.reggaeton AS `genre:reggaeton`,
    cgp.electropop AS `genre:electropop`,
    cgp.dance_pop AS `genre:dance_pop`,
    cgp.pop_rap AS `genre:pop_rap`,
    cgp.musica_mexicana AS `genre:musica_mexicana`,
    cgp.trap AS `genre:trap`,
    cgp.modern_rock AS `genre:modern_rock`,
    cgp.classic_rock AS `genre:classic_rock`,
    cgp.uk_pop AS `genre:uk_pop`,
    cgp.k_pop AS `genre:k_pop`,
    cgp.tropical_house AS `genre:tropical_house`,
    cgp.melodic_rap AS `genre:melodic_rap`,
    cgp.canadian_pop AS `genre:canadian_pop`,
    cgp.modern_bollywood AS `genre:modern_bollywood`
FROM charts_genre_popularity AS cgp
JOIN charts_daily_popularity AS cdp
    ON cgp.region = cdp.region AND cgp.date = cdp.date
JOIN charts_daily_sum AS cds
    ON cgp.region = cds.region AND cgp.date = cds.date
JOIN daily_country_weather AS dcw
    ON cgp.region = dcw.country AND cgp.date = dcw.date
JOIN wdi_interpolated AS wdi
    ON cgp.region = wdi.country AND cgp.date = wdi.date
ORDER BY cgp.region, cgp.date
""")

Processing query and saving to '/output/*'...


Execution time: 6.595 seconds


25/06/10 17:30:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+----------+-------------+----------------+------------------+-------------------+----------------------------+------------------+----------------------+--------------------------------+-------------------------------+-------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+-------------------+-------------------+---------------------+--------------------+--------------------+------------------+--------------------+-----------+--------------------+--------------------+--------------------+----------------------+
|  country|      date|daily_streams|daily_popularity|     temperature_c|   precipitation_mm|wdi:rural_population_percent|wdi:fertility_rate|wdi:gdp_per_capita_usd|wdi:mobile_subscriptions_per_100|wdi:refugee_population_promille|          genre:pop|           genre:rap|          genre:rock|          genre:edm|       genre:hip_hop|  genre:trap_latino|    genre:regg

In [9]:
spark.stop()